In [0]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import max

In [0]:
for fname in os.listdir('/dbfs/output_dir/2020-08-05/partition=Q'):
    print(fname)

_SUCCESS
_committed_1832146638374157953
_committed_7801791286934985880
_started_1832146638374157953
_started_7801791286934985880
part-00000-tid-7801791286934985880-af0cb05e-d70e-4754-bd04-23af6b391e9c-59-1.c000.snappy.parquet
part-00001-tid-7801791286934985880-af0cb05e-d70e-4754-bd04-23af6b391e9c-60-1.c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/2020-08-06/partition=Q'):
    print(fname)

_SUCCESS
_committed_122085719052758727
_committed_3573979185984521094
_started_122085719052758727
_started_3573979185984521094
part-00000-tid-122085719052758727-75234440-a77c-41df-9546-3e5b549545d8-64-1.c000.snappy.parquet
part-00001-tid-122085719052758727-75234440-a77c-41df-9546-3e5b549545d8-65-1.c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/2020-08-05/partition=T'):
    print(fname)

_SUCCESS
_committed_1832146638374157953
_committed_7801791286934985880
_started_1832146638374157953
_started_7801791286934985880
part-00000-tid-7801791286934985880-af0cb05e-d70e-4754-bd04-23af6b391e9c-59-2.c000.snappy.parquet
part-00001-tid-7801791286934985880-af0cb05e-d70e-4754-bd04-23af6b391e9c-60-2.c000.snappy.parquet

In [0]:
for fname in os.listdir('/dbfs/output_dir/2020-08-06/partition=T'):
    print(fname)

_SUCCESS
_committed_122085719052758727
_committed_3573979185984521094
_started_122085719052758727
_started_3573979185984521094
part-00000-tid-122085719052758727-75234440-a77c-41df-9546-3e5b549545d8-64-2.c000.snappy.parquet
part-00001-tid-122085719052758727-75234440-a77c-41df-9546-3e5b549545d8-65-2.c000.snappy.parquet

In [0]:
def applyLatest(tq):
    latest_arrival = tq.groupBy("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb").agg(max("arrival_tm").alias("latest"))
    return tq.join(latest_arrival, ["trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb"])

In [0]:
def clean(date, p):
    path = "/output_dir/{}/partition={}/".format(date, p) 
    filelist=os.listdir('/dbfs'+path)
    parquets=[]
    for name in filelist:
        if name.endswith('.parquet'):
            parquets.append(path+name)
    
    common_df = spark.read.parquet(*parquets)
    common_df.show()

    if p == 'T':
        trade = common_df.select("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb",
                                "arrival_tm", "trade_pr")
        corrected = applyLatest(trade)
        corrected.write.mode("append").parquet("output_dir/trade/date={}".format(date))
    else:
        quote = common_df.select("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb",
                                "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size")
        corrected = applyLatest(quote)
        corrected.write.mode("append").parquet("output_dir/quote/date={}".format(date))

In [0]:
#
clean('2020-08-05','Q')
clean('2020-08-05','T')
clean('2020-08-06','Q')
clean('2020-08-06','T')

+----------+--------+------+--------+--------------------+------------+-------------------+--------+---------+--------+---------+--------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm|trade_pr| bid_pr|bid_size| ask_pr|ask_size|
+----------+--------+------+--------+--------------------+------------+-------------------+--------+---------+--------+---------+--------+
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:34:...| 1|2020-08-05 09:30:00| null| 75.30255| 100| 75.35917| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:40:...| 2|2020-08-05 09:30:00| null| 77.20875| 100| 78.90918| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:50:...| 3|2020-08-05 09:30:00| null| 77.15973| 100| 77.33205| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:57:...| 4|2020-08-05 09:30:00| null|79.299774| 100| 80.08399| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:06:...| 5|2020-08-05 09:30:00| null|77.863495| 100| 78.30821| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| null| 78.74799| 100|80.329056| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:17:...| 7|2020-08-05 09:30:00| null| 77.16837| 100| 77.97598| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:23:...| 8|2020-08-05 09:30:00| null| 78.06551| 100| 78.80516| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:31:...| 9|2020-08-05 09:30:00| null| 76.69653| 100| 78.56943| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| null| 75.10474| 100|77.096565| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:54:...| 12|2020-08-05 09:30:00| null| 74.52148| 100| 75.66917| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:03:...| 13|2020-08-05 09:30:00| null| 75.16876| 100| 76.70698| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:12:...| 14|2020-08-05 09:30:00| null| 74.56633| 100|74.621376| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:19:...| 15|2020-08-05 09:30:00| null| 74.49803| 100|76.475914| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:27:...| 16|2020-08-05 09:30:00| null| 75.22889| 100|76.143684| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:35:...| 17|2020-08-05 09:30:00| null| 76.62792| 100| 76.99149| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:42:...| 18|2020-08-05 09:30:00| null|77.094666| 100|78.159676| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:51:...| 19|2020-08-05 09:30:00| null|78.335365| 100| 79.39139| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 12:04:...| 21|2020-08-05 09:30:00| null|75.867325| 100| 77.4598| 100|
2020-08-05| Q| SYMA| NYSE|2020-08-05 12:11:...| 22|2020-08-05 09:30:00| null| 75.35314| 100| 76.32488| 100|
+----------+--------+------+--------+--------------------+------------+-------------------+--------+---------+--------+---------+--------+
only showing top 20 rows

+----------+--------+------+--------+--------------------+------------+-------------------+---------+------+--------+------+--------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr|bid_pr|bid_size|ask_pr|ask_size|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+------+--------+------+--------+
2020-08-05| T| SYMB| NYSE|2020-08-05 16:48:...| 60|2020-08-05 09:30:00|35.762054| null| 371| null| null|
2020-08-05| T| SYMB| NYSE|2020-08-05 18:02:...| 70|2020-08-05 09:30:00|32.681087| null| 318| null| null|
2020-08-05| T| SYMB| NYSE|2020-08-05 19:08:...| 80|2020-08-05 09:30:00| 34.27408| null| 548| null| null|
2020-08-05| T| SYMB| NYSE|2020-08-05 20:17:...| 90|2020-08-05 09:30:00| 34.2493| null| 145| null| null|
2020-08-05| T| SYMB| NYSE|2020-08-05 21:27:...| 100|2020-08-05 09:30:00|33.956287| null| 369| null| null|
2020-08-05| T| SYMC| NYSE|2020-08-05 10:44:...| 10|2020-08-05 09:30:00|160.87872| null| 836| null| null|
2020-08-05| T| SYMC| NYSE|2020-08-05 11:58:...| 20|2020-08-05 09:30:00| 157.0271| null| 680| null| null|
2020-08-05| T| SYMC| NYSE|2020-08-05 13:10:...| 30|2020-08-05 09:30:00|157.25435| null| 1000| null| null|
2020-08-05| T| SYMC| NYSE|2020-08-05 14:32:...| 40|2020-08-05 09:30:00|